In [1]:
from baselines.common import set_global_seeds, tf_util as U
from baselines import bench
import gym, logging
import roboschool
from baselines import logger
from baselines.ppo1 import mlp_policy, pposgd_simple
from base_line_model.mlp import MlpPolicy_new
from base_line_model.PPO_agent import learning_agent

In [2]:
seed = 1
logger.configure()
U.make_session(num_cpu=16).__enter__()
set_global_seeds(seed)

Logging to /tmp/openai-2018-04-20-03-19-08-986306


# set up agents

In [3]:
env2 = gym.make('RoboschoolHopper-v1')
env2 = bench.Monitor(env2, logger.get_dir(),allow_early_resets=True)
env2.seed(seed)
gym.logger.setLevel(logging.WARN)
class pargm(object):
    def __init__(self):
        self.timesteps_per_actorbatch = 25000#25000 # timesteps per actor per update
        self.clip_param = 0.2 
        self.entcoeff = 0.0 # clipping parameter epsilon, entropy coeff
        self.optim_epochs = 10 
        self.optim_stepsize = 3e-4
        self.optim_batchsize = 64# optimization hypers
        self.gamma = 0.99
        self.lam = 0.95 # advantage estimation
        self.max_timesteps=10e4#1e10 
        self.max_episodes=0 
        self.max_iters=0 
        self.max_seconds=0  # time constraint
        self.callback=None # you can do anything in the callback, since it takes locals(), globals()
        self.adam_epsilon=1e-5
        self.schedule='linear' # annealing for stepsize parameters (epsilon and adam)


def policy_fn(name, ob_space, ac_space):
        return MlpPolicy_new(name=name, ob_space=ob_space, ac_space=ac_space,
            hid_size=64, num_hid_layers=2)
    
parg = pargm()
agent2 = learning_agent('pi2', env2, policy_fn, parg)

[2018-04-20 03:19:09,014] Making new env: RoboschoolHopper-v1


# Training

In [4]:
agent2.learn()

********** Iteration 0 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
   -0.0148581 |           0.0 |       37.7197 |    0.00731822 |       4.19116
   -0.0201327 |           0.0 |       11.0771 |     0.0134314 |       4.12614
   -0.0210902 |           0.0 |       10.0607 |     0.0148613 |       4.11216
   -0.0211594 |           0.0 |       9.81106 |     0.0150585 |       4.11231
   -0.0215619 |           0.0 |       9.65636 |     0.0153284 |        4.1105
   -0.0215118 |           0.0 |       9.55002 |     0.0156136 |       4.10882
   -0.0216761 |           0.0 |       9.48059 |     0.0153879 |       4.11452
   -0.0214674 |           0.0 |       9.43146 |      0.015769 |       4.11295
   -0.0220529 |           0.0 |       9.36947 |     0.0150825 |       4.11352
   -0.0226045 |           0.0 |       9.34058 |      0.015584 |        4.1135
Evaluating losses...
   -0.0230899 |             0 |        9.3273 |     0.0148194 |       4

# Evaluation

In [5]:
env = gym.make("RoboschoolHopper-v1")
for i in range(10):
    score = 0
    obs = env.reset()
    done = False
    itr = 0
    do = False
    while done == False:   
        a = agent2.action_ev(obs)
        obs, r, done, _ = env.step(a)
        if done:
            do = True

        score += r
    print(score)

28.465061497176066
28.017580000843736
32.81638739875635
38.022979164357814
28.100173767177328
27.09531361210538
30.17301668100699
28.748361785235463
29.95723484088521
28.024001243665403


In [6]:
agent2.save_data('hopper_mem')

In [7]:
#agent2.restore('hopper_mem')